In [1]:
pip install meteostat


Note: you may need to restart the kernel to use updated packages.


In [2]:
import meteostat

https://dev.meteostat.net/python/#installation

In [3]:
#Attempting to get to weather stations as a list: 
import requests

url = 'https://meteostat.p.rapidapi.com/stations/daily'
headers = {
    'x-rapidapi-host': 'meteostat.p.rapidapi.com',
    'x-rapidapi-key': '619929cb42dmshf6dd297298e1eddp168b76jsne44e4f04a2f'  # Replace {your_api_key} with your actual API key
}
params = {
    'station': '10637',
    'start': '2020-01-01',
    'end': '2020-01-01'
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    # Process the data as needed
    print(data)
else:
    print('Error:', response.status_code)


Error: 403


In [4]:
#Attempting to get to weather stations as DF 
import requests
import pandas as pd

url = 'https://meteostat.p.rapidapi.com/stations/daily'
headers = {
    'x-rapidapi-host': 'meteostat.p.rapidapi.com',
    'x-rapidapi-key': ''  # Replace {your_api_key} with your actual API key
}
params = {
    'station': '10637',
    'start': '2020-01-01',
    'end': '2020-01-11'
}

# Make the API request
response = requests.get(url, headers=headers, params=params)

# Extract JSON data from the response
json_data = response.json()

# Normalize the JSON data into a DataFrame
weather_df = pd.json_normalize(json_data['data'], 
                               record_path=None, 
                               meta=['meta'])

# Print or use the DataFrame as needed
weather_df


KeyError: 'data'

Follow this for API key: 
https://rapidapi.com/developer/authorization/default-application_9067818

In [ ]:
  url: 'https://meteostat.p.rapidapi.com/stations/meta',
  params: {id: '10637'},
  headers: {
    'X-RapidAPI-Key': '2a4c4f60fdmshff28636b9a3f8c0p18f5f5jsn3b317a6c373b',
    'X-RapidAPI-Host': 'meteostat.p.rapidapi.com'
  }

#Create location directory: 
location_directory = ["Sao Paulo",
          "Cape Town",
          "Havana"]

#Define root url
url = 'http://api.openweathermap.org/data/2.5/weather?'

#Define parameters to be added to url
parameters = {
    'appid': os.getenv('openweather_api_key'), 
    'units': 'metric',
    'lang': 'pt'}

#Define how the output will be displayed
weather_current_df = pd.DataFrame([])

#For each directory entry, define what should happen: 
for each_location in location_directory:
  #Pass on the parameters for each location:
  parameters["q"] = each_location #Check what this 'q' means 
  #The response is a REQUEST to fetch data, which is successful by following the url AND parameters:
  response = requests.get(url, parameters)
  #The output will be a response in javascript object notation, JSON format:
  weather_json_response = response.json()
  #Given the json format, next NORMALIZE it, so it can be outputted as a DATAFRAME: 
  weather_json_to_df = pd.json_normalize(weather_json_response, 
                                    sep="_", #naming convention, e.g. coord_lon, coord_lat
                                    record_path="weather", 
                                    meta=["id", #I can also leave out the meta completely 
                                          "name",
                                          "cod",
                                          "timezone",
                                          ["coord", "lon"], 
                                            ["coord", "lat"], 
                                            "base",
                                            ["main", "temp"],
                                            ["main", "feels_like"],
                                            ["main", "temp_min"],
                                            ["main", "temp_max"], 
                                            ["main", "pressure"], 
                                            ["main", "humidity"], 
                                            "visibility", 
                                            ["wind", "speed"], 
                                            ["wind", "deg"], 
                                            ["clouds", "all"], 
                                            "dt", 
                                            ["sys", "type"], 
                                            ["sys", "id"],
                                            ["sys", "country"],
                                            ["sys", "sunrise"],
                                            ["sys", "sunset"]], 
                                    record_prefix="weather_") #Insert this prefix in every column name
  #Concatenate the DFs: one was the normalized one from the JSON response, and the other one was the empty one created earlier
  weather_current_df = pd.concat([weather_current_df, weather_json_to_df], ignore_index=True)
weather_current_df

In [ ]:
import requests

def get_nearest_station(lat, lon):
    url = 'https://meteostat.p.rapidapi.com/stations/nearby'
    headers = {
        'x-rapidapi-host': 'meteostat.p.rapidapi.com',
        'x-rapidapi-key': '2a4c4f60fdmshff28636b9a3f8c0p18f5f5jsn3b317a6c373b'  # Replace with your actual API key
    }
    params = {
        'lat': lat,
        'lon': lon
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data['data'][0]['id'] if data['data'] else None

# Example usage
latitude = 38.9445
longitude = -77.4558
station_code = get_nearest_station(latitude, longitude)
print(station_code)  # This will print the nearest weather station code

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# Import Meteostat library
from meteostat import Stations

# Example usage
latitude = 38.9445
longitude = -77.4558

# Get nearby weather stations
stations = Stations()
stations = stations.nearby(latitude, longitude)
station = stations.fetch(1)

# Print DataFrame
print(station)

                               name country region    wmo  icao  latitude  \
id                                                                          
72403  Dulles International Airport      US     VA  72403  KIAD   38.9333   

       longitude  elevation          timezone hourly_start hourly_end  \
id                                                                      
72403     -77.45       95.0  America/New_York   1973-01-01 2024-05-07   

      daily_start  daily_end monthly_start monthly_end     distance  
id                                                                   
72403  1960-04-01 2024-12-30    1961-01-01  2022-01-01  1342.616722  


In [ ]:
IAD = Point(38.9445, -77.4558, 95)
BWI = Point(39.1774, -76.6684, 146)
PHL = Point(39.8721, -75.2431, 36)
EWR = Point(40.6895, -74.1745, 18)
JFK = Point(40.6413, -73.7781, 13)
LGA = Point(40.7769, -73.874, 21)

In [ ]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily

# Set time period
start = datetime(2010, 2, 1)
end = datetime(2010, 2, 28)

# Create Point for Vancouver, BC
DCA = Point(38.8521, -77.0377, 0)

# Get daily data for 2018
data = Daily(DCA, start, end)
data = data.fetch()
data
# Plot line chart including average, minimum and maximum temperature
#data.plot(y=['tavg', 'tmin', 'tmax'])
#plt.show()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2010-02-01,-0.8,-6.7,3.9,0.0,102.0,NaN,8.3,NaN,1026.6,NaN
2010-02-02,0.5,-2.8,5.0,5.1,51.0,NaN,6.8,NaN,1022.2,NaN
2010-02-03,2.4,-1.1,5.6,1.5,127.0,323.0,9.4,NaN,1020.1,NaN
2010-02-04,2.4,-0.6,5.0,0.0,25.0,NaN,12.2,NaN,1028.9,NaN
2010-02-05,1.0,0.0,2.8,19.6,0.0,NaN,12.2,NaN,1021.6,NaN
2010-02-06,-2.4,-6.1,0.0,18.5,330.0,7.0,25.9,NaN,1004.7,NaN
2010-02-07,-4.3,-8.9,0.0,0.0,457.0,316.0,13.3,NaN,1016.2,NaN
2010-02-08,-2.6,-6.1,1.1,0.0,356.0,292.0,13.7,NaN,1020.9,NaN
2010-02-09,-1.7,-5.6,2.2,9.7,330.0,NaN,6.1,NaN,1017.2,NaN


In [ ]:
#Attempting to get to hourly data 
import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables from .env file
load_dotenv()

headers = {
    'x-rapidapi-host': 'meteostat.p.rapidapi.com',
    'x-rapidapi-key': '619929cb42dmshf6dd297298e1eddp168b76jsne44e4f04a2f'  # Replace {your_api_key} with your actual API key
}

# Retrieve API key from environment variable
#api_key = os.getenv('x-rapid_api_key')

In [ ]:
parameters = {
     'appid': '2a4c4f60fdmshff28636b9a3f8c0p18f5f5jsn3b317a6c373b',
    'units': 'metric',
    'lat': '38.9554',
    'lon': '-77.4558',
    'start': '2010-02-06',
    'end': '2010-02-06',
      # Assuming this is the correct parameter name for the API key
    }

# Define the API endpoint and parameters
#curl --request GET \
url = 'https://meteostat.p.rapidapi.com/point/hourly' 

###?lat=38.9554&lon=-77.4558&start=2010-02-06&end=2010-02-6' 


#Define headers for the request
#headers = {
    #'x-rapidapi-host': 'meteostat.p.rapidapi.com',
    #'x-rapidapi-key': api_key
#}
#send request
r = requests.get(url, parameters)

#print(r)
# Iterate over each station
#for station_id in station_ids:
    # Define parameters for the current station
    #params = {
        #'station': station_id,
        #'start': '2010-02-01',
        #'end': '2010-02-28'
#}

weather_hourly = r.json()

print(weather_hourly)
#return data['data'][0]['id'] if data['data'] else None

# Make the API request
#response = requests.get(url, headers=headers, params=parameters)


# Extract JSON data from the response
#json_data = response.json()

# Normalize the JSON data into a DataFrame
#weather_hourly_df = pd.json_normalize(json_data['data'], 
                               #record_path=None, 
                               #meta=['meta'])

# Print or use the DataFrame as needed
#weather_hourly_df

{'message': 'Invalid API key. Go to https://docs.rapidapi.com/docs/keys for more info.'}


{'message': "Error in validating the request 77.4558' is not valid value for number"}


In [16]:
import requests

url = "https://meteostat.p.rapidapi.com/point/hourly"

querystring = {"lat":"38.9554","lon":"-77.4558","start":"2010-02-06","end":"2010-02-06","tz":"America/Toronto"}

headers = {
	"X-RapidAPI-Key": "619929cb4dmshf6dd297298e1eddp168b76jsne44e4f04a2f2",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'meta': {'generated': '2024-05-08 10:24:49', 'stations': ['72403', 'KJYO0', 'KHEF0', 'KHWY0']}, 'data': [{'time': '2010-02-06 00:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 2.5, 'snow': None, 'wdir': 60.0, 'wspd': 33.5, 'wpgt': None, 'pres': 1005.6, 'tsun': None, 'coco': None}, {'time': '2010-02-06 01:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 2.3, 'snow': None, 'wdir': 40.0, 'wspd': 29.5, 'wpgt': None, 'pres': 1005.4, 'tsun': None, 'coco': None}, {'time': '2010-02-06 02:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 3.0, 'snow': None, 'wdir': 30.0, 'wspd': 18.4, 'wpgt': None, 'pres': 1004.2, 'tsun': None, 'coco': None}, {'time': '2010-02-06 03:00:00', 'temp': -0.6, 'dwpt': -2.2, 'rhum': 89.0, 'prcp': 0.8, 'snow': None, 'wdir': 40.0, 'wspd': 29.5, 'wpgt': None, 'pres': 1001.6, 'tsun': None, 'coco': None}, {'time': '2010-02-06 04:00:00', 'temp': -1.1, 'dwpt': -2.2, 'rhum': 92.0, 'prcp': 1.5, 'snow': None, 'wdir': 10.0, 'wspd': 13.0, 'wpgt': None,

In [17]:
print(response)

<Response [200]>


In [8]:
response.headers

{'Date': 'Wed, 08 May 2024 10:15:44 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Cache-Control': 'public, must-revalidate, max-age=259200', 'Last-Modified': 'Wed, 08 May 2024 09:25:43 GMT', 'NEL': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'CF-Cache-Status': 'HIT', 'Vary': 'Accept-Encoding', 'CF-RAY': '8808b0b8cef118bd-FRA', 'X-Meteostat-Server': 'jasper', 'alt-svc': 'h3=":443"; ma=86400', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v4?s=Y3vzkzRJfIOfXnZdaqr6qigUsXlztfDb7qnZ%2FJ1Z1fGrLSSK101plZ4Di36fjNrIQHazXxXiUvqMgpMCxLN0D%2FN5HgZQhJI6SrQa8AEJETjawQumItCfmdaPecsI28lvRcC57Qe85Q%3D%3D"}],"group":"cf-nel","max_age":604800}', 'X-RateLimit-Requests-Limit': '500', 'X-RateLimit-Requests-Remaining': '497', 'X-RateLimit-Requests-Reset': '2675335', 'X-RateLimit-rapid-free-plans-hard-limit-Limit': '500000', 'X-RateLimit-rapid-free-plans-hard-limit-Remaini

In [18]:
response.content

b'{"meta":{"generated": "2024-05-08 10:24:49", "stations": ["72403", "KJYO0", "KHEF0", "KHWY0"]},"data":[{"time":"2010-02-06 00:00:00","temp":-0.6,"dwpt":-1.7,"rhum":92.0,"prcp":2.5,"snow":null,"wdir":60.0,"wspd":33.5,"wpgt":null,"pres":1005.6,"tsun":null,"coco":null},{"time":"2010-02-06 01:00:00","temp":-0.6,"dwpt":-1.7,"rhum":92.0,"prcp":2.3,"snow":null,"wdir":40.0,"wspd":29.5,"wpgt":null,"pres":1005.4,"tsun":null,"coco":null},{"time":"2010-02-06 02:00:00","temp":-0.6,"dwpt":-1.7,"rhum":92.0,"prcp":3.0,"snow":null,"wdir":30.0,"wspd":18.4,"wpgt":null,"pres":1004.2,"tsun":null,"coco":null},{"time":"2010-02-06 03:00:00","temp":-0.6,"dwpt":-2.2,"rhum":89.0,"prcp":0.8,"snow":null,"wdir":40.0,"wspd":29.5,"wpgt":null,"pres":1001.6,"tsun":null,"coco":null},{"time":"2010-02-06 04:00:00","temp":-1.1,"dwpt":-2.2,"rhum":92.0,"prcp":1.5,"snow":null,"wdir":10.0,"wspd":13.0,"wpgt":null,"pres":1002.7,"tsun":null,"coco":null},{"time":"2010-02-06 05:00:00","temp":-1.1,"dwpt":-2.8,"rhum":88.0,"prcp":2.

In [19]:
# Apply JSON decoder and save in weather_hh
weather_hourly = response.json()

# Print weather_hh
print(weather_hourly)

{'meta': {'generated': '2024-05-08 10:24:49', 'stations': ['72403', 'KJYO0', 'KHEF0', 'KHWY0']}, 'data': [{'time': '2010-02-06 00:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 2.5, 'snow': None, 'wdir': 60.0, 'wspd': 33.5, 'wpgt': None, 'pres': 1005.6, 'tsun': None, 'coco': None}, {'time': '2010-02-06 01:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 2.3, 'snow': None, 'wdir': 40.0, 'wspd': 29.5, 'wpgt': None, 'pres': 1005.4, 'tsun': None, 'coco': None}, {'time': '2010-02-06 02:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 3.0, 'snow': None, 'wdir': 30.0, 'wspd': 18.4, 'wpgt': None, 'pres': 1004.2, 'tsun': None, 'coco': None}, {'time': '2010-02-06 03:00:00', 'temp': -0.6, 'dwpt': -2.2, 'rhum': 89.0, 'prcp': 0.8, 'snow': None, 'wdir': 40.0, 'wspd': 29.5, 'wpgt': None, 'pres': 1001.6, 'tsun': None, 'coco': None}, {'time': '2010-02-06 04:00:00', 'temp': -1.1, 'dwpt': -2.2, 'rhum': 92.0, 'prcp': 1.5, 'snow': None, 'wdir': 10.0, 'wspd': 13.0, 'wpgt': None,

In [11]:
for key_value in weather_hourly.items():
    print(key + ':' + value)

NameError: name 'key' is not defined

In [20]:
type(weather_hourly)

dict

In [21]:
for key, value in weather_hourly.items():
    print(key + ':' , value)

meta: {'generated': '2024-05-08 10:24:49', 'stations': ['72403', 'KJYO0', 'KHEF0', 'KHWY0']}
data: [{'time': '2010-02-06 00:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 2.5, 'snow': None, 'wdir': 60.0, 'wspd': 33.5, 'wpgt': None, 'pres': 1005.6, 'tsun': None, 'coco': None}, {'time': '2010-02-06 01:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 2.3, 'snow': None, 'wdir': 40.0, 'wspd': 29.5, 'wpgt': None, 'pres': 1005.4, 'tsun': None, 'coco': None}, {'time': '2010-02-06 02:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 3.0, 'snow': None, 'wdir': 30.0, 'wspd': 18.4, 'wpgt': None, 'pres': 1004.2, 'tsun': None, 'coco': None}, {'time': '2010-02-06 03:00:00', 'temp': -0.6, 'dwpt': -2.2, 'rhum': 89.0, 'prcp': 0.8, 'snow': None, 'wdir': 40.0, 'wspd': 29.5, 'wpgt': None, 'pres': 1001.6, 'tsun': None, 'coco': None}, {'time': '2010-02-06 04:00:00', 'temp': -1.1, 'dwpt': -2.2, 'rhum': 92.0, 'prcp': 1.5, 'snow': None, 'wdir': 10.0, 'wspd': 13.0, 'wpgt': None, 'pres

In [22]:
weather_hourly

{'meta': {'generated': '2024-05-08 10:24:49',
  'stations': ['72403', 'KJYO0', 'KHEF0', 'KHWY0']},
 'data': [{'time': '2010-02-06 00:00:00',
   'temp': -0.6,
   'dwpt': -1.7,
   'rhum': 92.0,
   'prcp': 2.5,
   'snow': None,
   'wdir': 60.0,
   'wspd': 33.5,
   'wpgt': None,
   'pres': 1005.6,
   'tsun': None,
   'coco': None},
  {'time': '2010-02-06 01:00:00',
   'temp': -0.6,
   'dwpt': -1.7,
   'rhum': 92.0,
   'prcp': 2.3,
   'snow': None,
   'wdir': 40.0,
   'wspd': 29.5,
   'wpgt': None,
   'pres': 1005.4,
   'tsun': None,
   'coco': None},
  {'time': '2010-02-06 02:00:00',
   'temp': -0.6,
   'dwpt': -1.7,
   'rhum': 92.0,
   'prcp': 3.0,
   'snow': None,
   'wdir': 30.0,
   'wspd': 18.4,
   'wpgt': None,
   'pres': 1004.2,
   'tsun': None,
   'coco': None},
  {'time': '2010-02-06 03:00:00',
   'temp': -0.6,
   'dwpt': -2.2,
   'rhum': 89.0,
   'prcp': 0.8,
   'snow': None,
   'wdir': 40.0,
   'wspd': 29.5,
   'wpgt': None,
   'pres': 1001.6,
   'tsun': None,
   'coco': None},
 

In [29]:
import json
# Create json object: json_object
json_object = json.loads(response.content)

# Print json_object
json.dumps(json_object, indent = 3)

'{\n   "meta": {\n      "generated": "2024-05-08 10:24:49",\n      "stations": [\n         "72403",\n         "KJYO0",\n         "KHEF0",\n         "KHWY0"\n      ]\n   },\n   "data": [\n      {\n         "time": "2010-02-06 00:00:00",\n         "temp": -0.6,\n         "dwpt": -1.7,\n         "rhum": 92.0,\n         "prcp": 2.5,\n         "snow": null,\n         "wdir": 60.0,\n         "wspd": 33.5,\n         "wpgt": null,\n         "pres": 1005.6,\n         "tsun": null,\n         "coco": null\n      },\n      {\n         "time": "2010-02-06 01:00:00",\n         "temp": -0.6,\n         "dwpt": -1.7,\n         "rhum": 92.0,\n         "prcp": 2.3,\n         "snow": null,\n         "wdir": 40.0,\n         "wspd": 29.5,\n         "wpgt": null,\n         "pres": 1005.4,\n         "tsun": null,\n         "coco": null\n      },\n      {\n         "time": "2010-02-06 02:00:00",\n         "temp": -0.6,\n         "dwpt": -1.7,\n         "rhum": 92.0,\n         "prcp": 3.0,\n         "snow": nul

In [30]:
weather_hourly = pd.json_normalize(json_data[''], record_path=None, meta=['meta'])

KeyError: ''

In [31]:
pd.DataFrame(weather_hourly)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [24]:
weather_hourly_norm = pd.json_normalize(weather_hourly, sep="_")

# Print weather_hh_norm
weather_hourly_norm

,data,meta_generated,meta_stations
0,"[{'time': '2010-02-06 00:00:00', 'temp': -0.6,...",2024-05-08 10:24:49,"[72403, KJYO0, KHEF0, KHWY0]"


In [25]:
json.dumps(json_object, indent = 3)

NameError: name 'json' is not defined

In [26]:
for key, values in weather_hourly.items():
    print(key + ':', values)

meta: {'generated': '2024-05-08 10:24:49', 'stations': ['72403', 'KJYO0', 'KHEF0', 'KHWY0']}
data: [{'time': '2010-02-06 00:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 2.5, 'snow': None, 'wdir': 60.0, 'wspd': 33.5, 'wpgt': None, 'pres': 1005.6, 'tsun': None, 'coco': None}, {'time': '2010-02-06 01:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 2.3, 'snow': None, 'wdir': 40.0, 'wspd': 29.5, 'wpgt': None, 'pres': 1005.4, 'tsun': None, 'coco': None}, {'time': '2010-02-06 02:00:00', 'temp': -0.6, 'dwpt': -1.7, 'rhum': 92.0, 'prcp': 3.0, 'snow': None, 'wdir': 30.0, 'wspd': 18.4, 'wpgt': None, 'pres': 1004.2, 'tsun': None, 'coco': None}, {'time': '2010-02-06 03:00:00', 'temp': -0.6, 'dwpt': -2.2, 'rhum': 89.0, 'prcp': 0.8, 'snow': None, 'wdir': 40.0, 'wspd': 29.5, 'wpgt': None, 'pres': 1001.6, 'tsun': None, 'coco': None}, {'time': '2010-02-06 04:00:00', 'temp': -1.1, 'dwpt': -2.2, 'rhum': 92.0, 'prcp': 1.5, 'snow': None, 'wdir': 10.0, 'wspd': 13.0, 'wpgt': None, 'pres

In [45]:
weather_hourly_df = pd.json_normalize(weather_hourly, 
                                  sep="_", 
                                  record_path="data", 
                                  meta=["lon",
                                        "lat", 
                                        "temp", 
                                        "dwpt", 
                                        "prcp",
                                        "snow",
                                        "wispd",
                                        "prcp"],
                                        )
# Print weather_hh_df
print(weather_hourly_df)

KeyError: "Key 'lon' not found. To replace missing values of 'lon' with np.nan, pass in errors='ignore'"

In [42]:
weather_hourly.columns



AttributeError: 'dict' object has no attribute 'columns'

In [44]:
weather_hourly_df = pd.json_normalize(weather_hourly, 
                                      sep="_", 
                                      record_path="data", 
                                      meta=["stations", 
                                            "temp", 
                                            "dwpt", 
                                            "prcp",
                                            "snow",
                                            "wspd"]  
                                      )
# Print weather_hourly_df
print(weather_hourly_df)


KeyError: "Key 'stations' not found. To replace missing values of 'stations' with np.nan, pass in errors='ignore'"

In [46]:
weather_hourly_df = pd.json_normalize(weather_hourly, 
                                      sep="_", 
                                      record_path="data", 
                                      meta=["stations", 
                                            "lon", 
                                            "lat", 
                                            "temp", 
                                            "dwpt", 
                                            "prcp",
                                            "snow",
                                            "wspd"]  # Corrected the column name
                                      )
# Print weather_hourly_df
print(weather_hourly_df)


KeyError: "Key 'stations' not found. To replace missing values of 'stations' with np.nan, pass in errors='ignore'"

In [48]:
weather_hourly

{'meta': {'generated': '2024-05-08 10:24:49',
  'stations': ['72403', 'KJYO0', 'KHEF0', 'KHWY0']},
 'data': [{'time': '2010-02-06 00:00:00',
   'temp': -0.6,
   'dwpt': -1.7,
   'rhum': 92.0,
   'prcp': 2.5,
   'snow': None,
   'wdir': 60.0,
   'wspd': 33.5,
   'wpgt': None,
   'pres': 1005.6,
   'tsun': None,
   'coco': None},
  {'time': '2010-02-06 01:00:00',
   'temp': -0.6,
   'dwpt': -1.7,
   'rhum': 92.0,
   'prcp': 2.3,
   'snow': None,
   'wdir': 40.0,
   'wspd': 29.5,
   'wpgt': None,
   'pres': 1005.4,
   'tsun': None,
   'coco': None},
  {'time': '2010-02-06 02:00:00',
   'temp': -0.6,
   'dwpt': -1.7,
   'rhum': 92.0,
   'prcp': 3.0,
   'snow': None,
   'wdir': 30.0,
   'wspd': 18.4,
   'wpgt': None,
   'pres': 1004.2,
   'tsun': None,
   'coco': None},
  {'time': '2010-02-06 03:00:00',
   'temp': -0.6,
   'dwpt': -2.2,
   'rhum': 89.0,
   'prcp': 0.8,
   'snow': None,
   'wdir': 40.0,
   'wspd': 29.5,
   'wpgt': None,
   'pres': 1001.6,
   'tsun': None,
   'coco': None},
 